# GAN - FFHQ - 128×128 px
Generative Adversarial Network for generating images of faces from Flickr-Faces-HQ database - code for generating images.

Developed by Daniel Konečný

## Initialize
Defines the basic libraries and initializes global variables needed in all codes. Connects the code to data source - Google Drive.

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import imageio
import os
from tensorflow.keras.models import load_model
from PIL import Image, ImageDraw, ImageFont
from glob import glob

dataset = 'ffhq'
x_dimension = 128
y_dimension = 128
note = ''

project_name = f'{dataset}{x_dimension}x{y_dimension}{note}'
project_path = '/'
animation_path = f'{project_path}animations/'
image_path = f'{project_path}images/'
model_path = f'{project_path}models/'
weight_path = f'{project_path}weights/'

latent_dimension = 256

## Generating functions
All functions necessary for generating, initialize before.

In [0]:
def get_latent_points(sample_count=1):
	latents = np.empty((sample_count, latent_dimension))

	for latents_index in range(sample_count):
		randoms = np.random.normal(0, 1, latent_dimension)
		normalizer = np.sum(randoms**2)**0.5
		latent = randoms/normalizer
		latents[latents_index] = latent
	
	return latents


def get_images(epoch_number, latent_points):
    print(f'Creating images from epoch {epoch_number}...')
    model = load_model(f'{model_path}{project_name}_generator{epoch_number:04d}.h5',
                       compile=False)
    images = model.predict(latent_points)
    return images


def save_image(image_numpy, index, specifier="image"):
    print('Saving image...')
    image_numpy = np.squeeze(image_numpy, axis=0)
    image_numpy *= 255.0
    image_numpy = image_numpy.astype('uint8')
    image_pil = Image.fromarray(image_numpy)

    if specifier == "animation":
        image_pil = image_pil.convert('RGBA')

        color = (255, 255, 255)
        transparency = 0.7
        opacity = int(255 * transparency)
        overlay = Image.new('RGBA', image_pil.size, color+(0,))
        draw = ImageDraw.Draw(overlay)
        draw.rectangle(((x_dimension-28, y_dimension-13),
                        (x_dimension-2, y_dimension-2)),
                       fill=color+(opacity,))

        draw.text((x_dimension-27, y_dimension-14),
                  text=f'{index:04d}',
                  fill='black', font=font)

        image_pil = Image.alpha_composite(image_pil, overlay)
        image_pil = image_pil.convert('RGB')

    filename = f'{image_path}{project_name}_{specifier}{index:04d}.png'
    image_pil.save(filename)


def load_images(filename):
    print("Loading images...")
    filenames = glob(filename)
    return sorted(filenames)


def create_animation(filenames):
    print("Creating animation...")
    with imageio.get_writer(f'{animation_path}{project_name}_animation.gif',
                            mode='I') as writer:
        for i, filename in enumerate(filenames):
            image = imageio.imread(filename)
            writer.append_data(image)
        for i in range(20):
            image = imageio.imread(filename)
            writer.append_data(image)


def delete_images(filenames):
    print("Deleting images...")
    for i, filename in enumerate(filenames):
        os.remove(filename)


def create_grid(images, image_grid_size):
	print("Creating image grid...")
	for grid_index in range(image_grid_size * image_grid_size):
		plt.subplot(image_grid_size, image_grid_size, 1 + grid_index)
		plt.axis('off')
		plt.imshow(images[grid_index, :, :, :])
	plt.savefig(f'{image_path}{project_name}_grid.png',
                bbox_inches='tight', pad_inches=0.2, dpi=180)

## Create Generator Models
Create generator models from saved weights. Set number of already created epochs, number of done epochs with saved weights and launch.

In [0]:
from tensorflow.keras import layers

finished_epochs = 700
epoch_count = 700


def get_generator():
	generator = tf.keras.Sequential()
	
	generator.add(layers.Dense(x_dimension//32 * y_dimension//32 * 256, input_dim=latent_dimension))
	generator.add(layers.LeakyReLU(alpha=0.2))
	generator.add(layers.Reshape((x_dimension//32, y_dimension//32, 256)))
	assert generator.output_shape == (None, x_dimension//32, y_dimension//32, 256)

	generator.add(layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//16, y_dimension//16, 256)

	generator.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//8, y_dimension//8, 128)

	generator.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//4, y_dimension//4, 128)
 
	generator.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//2, y_dimension//2, 64)
 
	generator.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension, y_dimension, 32)
 
	generator.add(layers.Conv2D(3, (1, 1), activation='sigmoid', padding='same'))
	assert generator.output_shape == (None, x_dimension, y_dimension, 3)

	return generator


generator = get_generator()

print("Creating generator models...")
for epoch_index in range(finished_epochs, epoch_count):
    generator.load_weights(f'{weight_path}{project_name}_generator{epoch_index+1:04d}.h5')
    generator.save(f'{model_path}{project_name}_generator{epoch_index+1:04d}.h5')
    print(f'- Model {epoch_index+1} created.')

## Create a Set of Images
Set the epoch index of the used model, first index of generating, number of wanted images and launch for generating.

In [0]:
epoch_number = 700
first_index = 0
image_count = 50

for image_index in range(first_index, first_index+image_count):
    latent_point = get_latent_points()
    image = get_images(epoch_number, latent_point)
    save_image(image, image_index)

## Create Training Progress Animation
Creates animation showing the progress of training when given the same point in the latent space. Set the final epoch index of the model and launch.

**Requires saved models that are not part of the submitted files because of their size. Therefore, it is necessary to train the network individually.**

In [0]:
epoch_count = 700

latent_point = get_latent_points()

for epoch_index in range(epoch_count):
    # First the animation goes slow for big steps in training,
    # then fast when the changes get smaller.
    if epoch_index < 50 or \
        epoch_index < 200 and (epoch_index + 1) % 5 == 0 or \
        epoch_index < 1000 and (epoch_index + 1) % 10 == 0 or \
        (epoch_index + 1) % 50 == 0:
        image = get_images(epoch_index+1, latent_point)
        save_image(image, epoch_index+1, "animation")

filenames = load_images(f'{image_path}{project_name}_animation*.png')
create_animation(filenames)
delete_images(filenames)

## Create Training Progress Images
Creates images from one latent and model but in different parts of training. Set the epoch indices of the model and launch.


In [0]:
epoch_numbers = [10, 50, 100, 700]

latent_point = get_latent_points()

for epoch_index in epoch_numbers:
    image = get_images(epoch_index, latent_point)
    save_image(image, epoch_index, "progress")

## Create a Grid of Final Images
Creates n×n grid of images generated from given epoch. Set the epoch index of the model, size of the square grid and launch.

In [0]:
epoch_number = 700
image_grid_size = 3

latent_points = get_latent_points(image_grid_size*image_grid_size)
images = get_images(epoch_number, latent_points)
create_grid(images, image_grid_size)